In [33]:
import torch
from torch_geometric.data import Data
from itertools import combinations

train_ingr_id = '../utils/data/train_labels_ingr_id.csv'
val_ingr_id = '../utils/data/val_labels_ingr_id.csv'
# test_ingr_id = './test_labels_ingr_id.csv'
import pandas as pd

train_df = pd.read_csv(train_ingr_id)
val_df = pd.read_csv(val_ingr_id)
train_df = pd.concat([train_df, val_df], ignore_index=True)
# val_df = pd.read_csv(val_ingr_id)
# test_df = pd.read_csv(test_ingr_id)

# combine the train, validation, and test DataFrames
# df = pd.concat([train_df, val_df, test_df], ignore_index=True)
df = train_df
print(len(df))

num_ingredients = len(df.columns) - 2  # Subtract 2 for 'dish_id' and 'image_path' columns

# df structure
# dish_id, ingredient 1, ingredient 2, ingredient 3, ingredient 4, ingredient 5, ingredient 6, ingredient 7, ingredient 8, ingredient 9, ingredient 10, ...
# x, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0
ingr_list = []
# for each dish, add the index of the ingredients where the value is 1 to the list
# e.g. dish_1 = [1, 3], dish_2 = [1, 4], dish_3 = [2, 3], ...
# ingr_list = [[1, 3], [1, 4], [2, 3], ...]
# Iterate over each row of the DataFrame
for _, row in df.iterrows():
    # We assume that the first column is 'dish_id' and the remaining columns represent ingredients.
    # Extract the values starting from the second column onward (i.e., ingredient columns)
    # last column is the image_path, so we exclude it
    ingredient_values = row.iloc[1:-1]
    
    # Find the indices (0-based) of the ingredients where the value is 1
    selected_indices = [i for i, val in enumerate(ingredient_values) if val == 1]
    
    # Append the list of selected ingredient indices for this dish
    ingr_list.append(selected_indices)

print(df.iloc[0])
# print the second row of the DataFrame
print(ingr_list[0])

2958
id                       dish_1562699612
brown rice                           1.0
quinoa                               1.0
olive oil                            1.0
carrot                               0.0
                            ...         
chilaquiles                          0.0
pasta salad                          0.0
balsamic vinegar                     0.0
toast                                0.0
img_indx            dish_1562699612.jpeg
Name: 0, Length: 201, dtype: object
[0, 1, 2]


In [34]:
import numpy as np
adj_matrix = np.zeros((num_ingredients, num_ingredients), dtype=np.int32)

for dish in ingr_list:
    for i, j in combinations(dish, 2):
        adj_matrix[i, j] += 1
        adj_matrix[j, i] += 1

adj_matrix

array([[  0,  12, 201, ...,   0,   0,   0],
       [ 12,   0, 124, ...,   0,   0,   0],
       [201, 124,   0, ...,   1,   1,   0],
       ...,
       [  0,   0,   1, ...,   0,   0,   0],
       [  0,   0,   1, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int32)

In [35]:
# print out the top 10 paired ingredients in bert with the highest cosine similarity
# bert
import torch

# load the ingredient embeddings
ingredient_embeddings = torch.load("./ingredient_embeddings_gat_512.pt")
bert_embeddings = torch.load("./ingredient_embeddings_bert.pt")

import pandas as pd
ing_id = "../utils/data/test_labels_ingr_id.csv"

df = pd.read_csv(ing_id)
ingr_name = df.columns[1:-1].to_list()

cos = torch.nn.CosineSimilarity(dim=0)

ingr_name_dict = {}
ingr_id_dict = {}

for i in range(10):
    for j in range(i+1, len(bert_embeddings)):
        sim = cos(bert_embeddings[i], bert_embeddings[j])
        ingr_name_dict[(ingr_name[i], ingr_name[j])] = sim.item()
        ingr_id_dict[(i, j)] = sim.item()
        
sorted_sim_dict = sorted(ingr_name_dict.items(), key=lambda x: x[1], reverse=True)
sorted_id_dict = sorted(ingr_id_dict.items(), key=lambda x: x[1], reverse=True)

top_100 = sorted_id_dict[:100]
for i in range(100):
    print(sorted_sim_dict[i])

(('quinoa', 'jicama'), 0.9840413331985474)
(('berries', 'corn'), 0.9838709831237793)
(('berries', 'grapes'), 0.9820185899734497)
(('berries', 'wine'), 0.9818021655082703)
(('carrot', 'wine'), 0.9810669422149658)
(('berries', 'sugar'), 0.9809611439704895)
(('berries', 'milk'), 0.980205237865448)
(('berries', 'tuna'), 0.979519784450531)
(('brown rice', 'white rice'), 0.978987991809845)
(('berries', 'squash'), 0.9789713025093079)
(('berries', 'garlic'), 0.9780934453010559)
(('berries', 'chicken'), 0.977954089641571)
(('carrot', 'chicken'), 0.9777261018753052)
(('carrot', 'tuna'), 0.9777044057846069)
(('berries', 'butter'), 0.9775931239128113)
(('carrot', 'ginger'), 0.9765673279762268)
(('berries', 'orange'), 0.9763238430023193)
(('berries', 'syrup'), 0.975771963596344)
(('quinoa', 'pesto'), 0.9757385849952698)
(('berries', 'chili'), 0.9755913615226746)
(('berries', 'pizza'), 0.9741999506950378)
(('carrot', 'pizza'), 0.9735509157180786)
(('carrot', 'tofu'), 0.9734382629394531)
(('berries',

/tmp/ipykernel_3938441/401531389.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ingredient_embeddings = torch.load("./ingredient_embeddings_gat_512.pt")
/tmp/ipykernel_

In [36]:
# for the top 100 paired ingredients, print out the number of co-occurrences
never_appear = 0

for pair in top_100:
    if adj_matrix[pair[0][0], pair[0][1]] == 0:
        never_appear += 1
        print(ingr_name[pair[0][0]], ingr_name[pair[0][1]], pair[1], top_100.index(pair), pair[0][0], pair[0][1])

print(never_appear)

quinoa jicama 0.9840413331985474 0 1 140
berries corn 0.9838709831237793 1 6 139
berries wine 0.9818021655082703 3 6 116
berries sugar 0.9809611439704895 5 6 136
berries tuna 0.979519784450531 7 6 118
berries squash 0.9789713025093079 9 6 45
berries chicken 0.977954089641571 11 6 77
berries butter 0.9775931239128113 14 6 126
berries syrup 0.975771963596344 17 6 178
quinoa pesto 0.9757385849952698 18 1 179
berries chili 0.9755913615226746 19 6 63
berries cookies 0.9731311202049255 23 6 129
carrot milk 0.9730768799781799 24 3 10
berries basil 0.9722170233726501 26 6 66
berries apple 0.9709581136703491 30 6 114
berries rosemary 0.9709539413452148 31 6 75
berries white wine 0.970842182636261 32 6 125
berries brown sugar 0.9703260660171509 33 6 181
berries hominy 0.9684455394744873 35 6 106
berries ginger 0.9684008955955505 36 6 80
carrot cookies 0.9680901169776917 38 3 129
berries mushroom 0.9663424491882324 47 6 82
berries steak 0.9659769535064697 49 6 81
berries tofu 0.9654885530471802 5

In [37]:
# in the co-occurrence matrix, find the top 10 ingredient that co-occur the most with the bacon
bacon_idx = 9
bacon_cooccur = adj_matrix[bacon_idx]
sorted_bacon_cooccur = sorted(enumerate(bacon_cooccur), key=lambda x: x[1], reverse=True)

for i in range(10):
    print(ingr_name[sorted_bacon_cooccur[i][0]], sorted_bacon_cooccur[i][1])


olive oil 48
scrambled eggs 48
salt 36
berries 26
vinegar 24
garlic 23
cucumbers 22
mustard 18
sweet potato 18
yam 16


In [38]:
# print out the similarity between bacon and olive oil
bacon_idx = 9
ingred_list = df.columns[1:-1].to_list()
olive_oil_idx = ingred_list.index("olive oil")
print(ingr_name[bacon_idx], ingr_name[olive_oil_idx], adj_matrix[bacon_idx, olive_oil_idx])

# bert embedding for bacon and olive oil
print(cos(bert_embeddings[bacon_idx], bert_embeddings[olive_oil_idx]))


bacon olive oil 48
tensor(0.6193)


In [42]:
# measure the similarity between the bacon and olive oil in ingredient embeddings
print(cos(ingredient_embeddings[bacon_idx], ingredient_embeddings[ingred_list.index("cereal")]))
print(cos(ingredient_embeddings[bacon_idx], ingredient_embeddings[olive_oil_idx]))

tensor(-0.1339)
tensor(0.6027)


In [44]:
ingr_name_dict = {}
ingr_id_dict = {}

for i in range(10):
    for j in range(i+1, len(ingredient_embeddings)):
        sim = cos(ingredient_embeddings[i], ingredient_embeddings[j])
        ingr_name_dict[(ingr_name[i], ingr_name[j])] = sim.item()
        ingr_id_dict[(i, j)] = sim.item()
        
sorted_sim_dict = sorted(ingr_name_dict.items(), key=lambda x: x[1], reverse=True)
sorted_id_dict = sorted(ingr_id_dict.items(), key=lambda x: x[1], reverse=True)

top_100 = sorted_id_dict[:100]

# for the top 100 paired ingredients, print out the number of co-occurrences
never_appear = 0

for pair in top_100:
    if adj_matrix[pair[0][0], pair[0][1]] < 1:
        never_appear += 1

print(never_appear)

13
